In [2]:
import pandas as pd
import numpy as np
import os 
import re
import operator
import nltk 
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.util import elementtree_indent
import operator
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

[nltk_data] Error loading punkt: <urlopen error [WinError 10060] A
[nltk_data]     connection attempt failed because the connected party
[nltk_data]     did not properly respond after a period of time, or
[nltk_data]     established connection failed because connected host
[nltk_data]     has failed to respond>
[nltk_data] Error loading wordnet: <urlopen error [WinError 10060] A
[nltk_data]     connection attempt failed because the connected party
[nltk_data]     did not properly respond after a period of time, or
[nltk_data]     established connection failed because connected host
[nltk_data]     has failed to respond>
[nltk_data] Error loading averaged_perceptron_tagger: <urlopen error
[nltk_data]     [WinError 10060] A connection attempt failed because
[nltk_data]     the connected party did not properly respond after a
[nltk_data]     period of time, or established connection failed
[nltk_data]     because connected host has failed to respond>
[nltk_data] Error loading stopwords: <

False

In [3]:
nltk.download('punkt')

[nltk_data] Error loading punkt: <urlopen error [WinError 10060] A
[nltk_data]     connection attempt failed because the connected party
[nltk_data]     did not properly respond after a period of time, or
[nltk_data]     established connection failed because connected host
[nltk_data]     has failed to respond>


False

In [4]:
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
ps = PorterStemmer()
import string

In [5]:
import json
from pandas.io.json import json_normalize

with open("captions/annotations/captions_val2014.json") as json_data:
    data = json.load(json_data)

anno_df = pd.DataFrame(data['annotations'])
image_df = pd.DataFrame(data['images'])

In [1]:
print(anno_df.head(10))

NameError: name 'anno_df' is not defined

In [7]:
class inverted_index:
    def __init__(self):
        url = 'captions/annotations/captions_val2014.json'
        anno_df, image_df = self.get_dataset(url)
        anno_df['caption'] = anno_df['caption'].apply(self.text_preprocess)
        self.index = self.generate_inverted_index(anno_df["caption"].head(10))
        self.index = self.to_df(self.index)

    def get_dataset(self, path):
        with open("captions/annotations/captions_val2014.json") as json_data:
          data = json.load(json_data)
        anno_df = pd.DataFrame(data['annotations'])
        image_df = pd.DataFrame(data['images'])
        return anno_df, image_df

    def text_preprocess(self, text):
        nltk_english_stopwords = stopwords.words('english')
        trans = str.maketrans('', '', string.punctuation)
        text = text.translate(trans)
        text = text.lower()
        cleaned_text = ""
        for word in text.split():
            if word not in nltk_english_stopwords:
                cleaned_text += word + " " 
        return cleaned_text 
    
    def generate_inverted_index(self, data: list):
        inv_idx_dict = {}
        for index, doc_text in enumerate(data):
            words = word_tokenize(doc_text)
            for word in words:
                if  ps.stem(word) not in inv_idx_dict.keys():
                    inv_idx_dict[ ps.stem(word)] = [index]
                elif index not in inv_idx_dict[ ps.stem(word)]:
                    inv_idx_dict[ ps.stem(word)].append(index)
        return inv_idx_dict
    
    def to_df(self, dict):
        df = pd.DataFrame();
        v = []
        i = 0
        for key, values in dict.items():
            df.loc[i,'key'] = key
            strval = ''
            for value in values:
                strval += str(value) + ' '
            df.loc[i,'values'] = strval
            i = i +1
        return df
    



In [8]:
inv_index = inverted_index()
index = inv_index.index
print(index)


          key    values
0      bicycl        0 
1     replica        0 
2       clock      0 9 
3       front      0 1 
4       wheel        0 
5       black        1 
6       honda        1 
7   motorcycl        1 
8        park      1 3 
9       garag        1 
10       room        2 
11       blue  2 6 7 8 
12       wall    2 7 8 
13      white    2 6 7 
14       sink      2 7 
15       door        2 
16        car        3 
17       seem        3 
18      illeg        3 
19     behind        3 
20      legal        3 
21       larg        4 
22    passeng        4 
23    airplan        4 
24        fli        4 
25        air        4 
26        gol        5 
27      plane        5 
28       take        5 
29     partli        5 
30     cloudi        5 
31        sky        5 
32      color        6 
33     scheme        6 
34      small        6 
35   bathroom    6 7 8 
36    lifesav        7 
37       boat        8 
38      theme        8 
39       life        8 
40    preserv   

In [9]:
class engine:
    def __init__(self, index: pd.DataFrame):
        self.vocabulary = list(index['key'])
        self.tfidf = TfidfVectorizer(vocabulary=self.vocabulary)
        self.tfidf.fit(index['key'])
        self.tfidf_tran = self.tfidf.transform(index['key'])
        
    
    def gen_vector_T(self, tokens):
        Q = np.zeros((len(self.vocabulary)))    
        x= self.tfidf.transform(tokens)
        for token in tokens[0].split(','):
            try:
                ind = self.vocabulary.index(token)
                Q[ind]  = x[0, self.tfidf.vocabulary_[token]]
            except:
                pass
        return Q

    def cosine_sim(self,a, b):
        cos_sim = np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))
        return cos_sim

    def search(self, query, index):
        tokens = word_tokenize(str(query))
        clean = []
        clean.append(tokens)
        query_vector = self.gen_vector_T(tokens)
        d_cosines = []
        for d in self.tfidf_tran.A:
            d_cosines.append(self.cosine_sim(query_vector, d))
        out = np.array(d_cosines).argsort()[:][::-1]
        a = pd.DataFrame()
        for i,ind in enumerate(out):
            a.loc[i,'index'] = str(ind)
            a.loc[i,'val'] = list(index['values'])[ind]
        return a



In [10]:
e = engine(index)
res = e.search('white ', index)
print(res)


   index       val
0     13    2 6 7 
1     42        9 
2     10        2 
3     18        3 
4     17        3 
5     16        3 
6     15        2 
7     14      2 7 
8     12    2 7 8 
9     11  2 6 7 8 
10     9        1 
11    20        3 
12     8      1 3 
13     7        1 
14     6        1 
15     5        1 
16     4        0 
17     3      0 1 
18     2      0 9 
19     1        0 
20    19        3 
21    21        4 
22    41        9 
23    32        6 
24    40        8 
25    39        8 
26    38        8 
27    37        8 
28    36        7 
29    35    6 7 8 
30    34        6 
31    33        6 
32    31        5 
33    22        4 
34    30        5 
35    29        5 
36    28        5 
37    27        5 
38    26        5 
39    25        4 
40    24        4 
41    23        4 
42     0        0 


In [24]:
from PIL import Image

In [51]:
!pip install opencv-python

In [52]:
import cv2

In [74]:
class image_retrive:
    def __init__(self, res, df):
        self.res = res
        self.ind = self.get_index(res)
        self.paths = self.get_images(self.ind, df)

    def get_index(self, res):
        indices = res['val'][0].split()
        indint = []
        for i in indices:
            indint.append(int(i))
        print(indint)
        return indint
    
    def get_images(self, indices, df):
        img_ids = []
        paths = []
        #print(df['image_id'][int(indices[0])])
        for i in indices:
            
            img_ids.append(df['image_id'][i])
        for i in img_ids:
            print(i)
            paths.append('val2014\\val2014\COCO_val2014_'+ '0'*(12 - len(str(i))) + str(i) +'.jpg')
        return paths

    def display_images(self):
        for path in self.paths:
            print(path)
            img = cv2.imread(path, cv2.IMREAD_COLOR)
            cv2.imshow(';;',img)
        
        

In [75]:
ir = image_retrive(res, anno_df)
ir.display_images()

[2, 6, 7]
322141
322141
322141
val2014\val2014\COCO_val2014_000000322141.jpg
val2014\val2014\COCO_val2014_000000322141.jpg
val2014\val2014\COCO_val2014_000000322141.jpg


: 